In [4]:
import pandas as pd
import os

In [5]:
dataset_path = "../dataset_mare.csv"
df = pd.read_csv(dataset_path)

print("Dataset carregado:")
print(df.head(10))

Dataset carregado:
   latitude  longitude                                          previsoes  \
0   -3.8333   -32.4033  [{'dia': 'QUA', 'data': '01/01/2025', 'hora1':...   
1   -1.6917   -50.4833  [{'dia': 'QUA', 'data': '01/01/2025', 'hora1':...   
2   -1.6917   -50.4833  [{'dia': 'QUA', 'data': '01/01/2025', 'hora1':...   
3  -25.4600   -48.1567  [{'dia': 'QUA', 'data': '01/01/2025', 'hora1':...   
4       NaN        NaN                                                NaN   
5       NaN        NaN                                                NaN   
6       NaN        NaN                                                NaN   
7       NaN        NaN                                                NaN   
8       NaN        NaN                                                NaN   
9       NaN        NaN                                                NaN   

                                 praia  dia        data  hora1  altura1  \
0   arquipelago_de_fernando_de_noronha    0         NaN   

In [ ]:
print("\nTipos de dados:")
print(df.dtypes)
print("\nValores ausentes por coluna:")
print(df.isnull().sum())


Tipos de dados:
latitude            float64
longitude           float64
previsoes            object
praia                object
dia                   int64
data         datetime64[ns]
hora1               float64
altura1             float64
hora2               float64
altura2              object
hora3               float64
altura3              object
hora4               float64
altura4              object
0                   float64
mes                   int64
ano                   int64
dtype: object

Valores ausentes por coluna:
latitude     12410
longitude    12410
previsoes    12410
praia            0
dia              0
data          7521
hora1            7
altura1          7
hora2            7
altura2          7
hora3            7
altura3          7
hora4            7
altura4          7
0            12417
mes              0
ano              0
dtype: int64


In [ ]:
df['data'] = pd.to_datetime(df['data'], errors='coerce')

altura_cols = ['altura1','altura2','altura3','altura4']
for col in altura_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

hora_cols = ['hora1','hora2','hora3','hora4']
for col in hora_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=['data'])

records = []
for idx, row in df.iterrows():
    for i in range(1,5):
        hora = row[f'hora{i}']
        altura = row[f'altura{i}']
        if pd.notna(hora) and pd.notna(altura):
            hora_str = f"{int(hora):04d}" if hora < 10000 else str(int(hora))
            hora_formatada = f"{hora_str[:2]}:{hora_str[2:]}"
            records.append({
                'praia': row['praia'],
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'data': row['data'],
                'hora': hora_formatada,
                'altura': altura
            })

df_normalizado = pd.DataFrame(records)

df_normalizado[['latitude','longitude']] = df_normalizado.groupby('praia')[['latitude','longitude']].ffill()
df_normalizado['hora_decimal'] = df_normalizado['hora'].apply(lambda x: int(x.split(':')[0]) + int(x.split(':')[1])/60)
df_normalizado = df_normalizado[pd.to_numeric(df_normalizado['altura'], errors='coerce').notna()]

df_normalizado['mes'] = df_normalizado['data'].dt.month
df_normalizado['ano'] = df_normalizado['data'].dt.year
df_normalizado['dia_mes'] = df_normalizado['data'].dt.day
df_normalizado['semana_ano'] = df_normalizado['data'].dt.isocalendar().week
df_normalizado['dia_da_semana'] = df_normalizado['data'].dt.day_name()

print("\nDataset normalizado:")
print(df_normalizado.head(10))



✅ Dataset normalizado pronto para previsão:
         praia  latitude  longitude       data   hora  altura  hora_decimal  \
0  barra_norte       0.0        0.0 2025-01-01  04:04    0.04      4.066667   
1  barra_norte       0.0        0.0 2025-01-01  10:27    3.41     10.450000   
2  barra_norte       0.0        0.0 2025-01-01  16:21    0.62     16.350000   
3  barra_norte       0.0        0.0 2025-01-01  22:34    3.52     22.566667   
4  barra_norte       0.0        0.0 2025-02-01  05:14    0.11      5.233333   
5  barra_norte       0.0        0.0 2025-02-01  11:51    3.48     11.850000   
6  barra_norte       0.0        0.0 2025-02-01  17:36    0.61     17.600000   
7  barra_norte       0.0        0.0 2025-02-01  23:59    3.38     23.983333   
8  barra_norte       0.0        0.0 2025-03-01  04:51   -0.03      4.850000   
9  barra_norte       0.0        0.0 2025-03-01  11:08    3.50     11.133333   

   mes   ano  dia_mes  semana_ano dia_da_semana  
0    1  2025        1           1  

In [ ]:
nao_nulos = df_normalizado[['latitude', 'longitude']].notna().sum()
print(nao_nulos)

latitude     18959
longitude    18959
dtype: int64


In [ ]:
df_normalizado.to_csv("../public/dataset_mare_tratado.csv", index=False)
print("Dataset normalizado salvo em dataset_mare_tratado.csv")

✅ Dataset normalizado salvo em dataset_mare_tratado.csv
